In [3]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
from afinn import Afinn
from pattern.en import sentiment
nltk.download('vader_lexicon')
nltk.download('punkt')
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from keras.layers import BatchNormalization
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from keras.layers import Embedding, GRU, Dense, concatenate, Input
from keras.regularizers import l1_l2
from keras.models import Sequential
import tensorflow.compat.v1 as tf

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/faizelahmed/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/faizelahmed/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
df=pd.read_csv('truthseeker_emotions.csv')

In [5]:
df.head(3)

,Unnamed: 0,author,statement,target,BinaryNumTarget,manual_keywords,tweet,5_label_majority_answer,3_label_majority_answer,anger,disgust,fear,joy,neutral,sadness,surprise
0,0,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",@POTUS Biden Blunders - 6 Month Update\n\nInfl...,Mostly Agree,Agree,0,0,0,0,0,1,0
1,1,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,NO MAJORITY,Agree,0,0,0,0,0,1,0
2,2,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",THE SUPREME COURT is siding with super rich pr...,Agree,Agree,0,0,0,0,0,1,0


In [6]:
df['textblob'] = df['statement'].apply(lambda x: TextBlob(x).sentiment.polarity)
afinn = Afinn()
df['afinn'] = df['statement'].apply(lambda x: afinn.score(x))
df['pattern'] = df['statement'].apply(lambda x: sentiment(x)[0])
analyzer = SentimentIntensityAnalyzer()

df['vader'] = df['statement'].apply(lambda x: analyzer.polarity_scores(x)['compound'])


In [7]:
df.head(3)

,Unnamed: 0,author,statement,target,BinaryNumTarget,manual_keywords,tweet,5_label_majority_answer,3_label_majority_answer,anger,disgust,fear,joy,neutral,sadness,surprise,textblob,afinn,pattern,vader
0,0,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",@POTUS Biden Blunders - 6 Month Update\n\nInfl...,Mostly Agree,Agree,0,0,0,0,0,1,0,0.0,-1.0,0.0,-0.6908
1,1,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,NO MAJORITY,Agree,0,0,0,0,0,1,0,0.0,-1.0,0.0,-0.6908
2,2,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",THE SUPREME COURT is siding with super rich pr...,Agree,Agree,0,0,0,0,0,1,0,0.0,-1.0,0.0,-0.6908


In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense

In [9]:
selected_columns = ['statement', 'BinaryNumTarget', 'anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise',
                    'afinn', 'pattern', 'vader', 'textblob']

In [10]:
data = df[selected_columns]


In [11]:
data

,statement,BinaryNumTarget,anger,disgust,fear,joy,neutral,sadness,surprise,afinn,pattern,vader,textblob
0,End of eviction moratorium means millions of A...,1.0,0,0,0,0,0,1,0,-1.0,0.0,-0.6908,0.0
1,End of eviction moratorium means millions of A...,1.0,0,0,0,0,0,1,0,-1.0,0.0,-0.6908,0.0
2,End of eviction moratorium means millions of A...,1.0,0,0,0,0,0,1,0,-1.0,0.0,-0.6908,0.0
3,End of eviction moratorium means millions of A...,1.0,0,0,0,0,0,1,0,-1.0,0.0,-0.6908,0.0
4,End of eviction moratorium means millions of A...,1.0,0,0,0,0,0,1,0,-1.0,0.0,-0.6908,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
134193,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,1,0,0,0,0,0,0,0.0,0.0,-0.3182,0.0
134194,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,1,0,0,0,0,0,0,0.0,0.0,-0.3182,0.0
134195,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,1,0,0,0,0,0,0,0.0,0.0,-0.3182,0.0
134196,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,1,0,0,0,0,0,0,0.0,0.0,-0.3182,0.0


In [12]:
df1=df

In [13]:
duplicates = df1[df1.duplicated(subset=['statement'], keep=False)].sum()

# Display the duplicate rows
print(duplicates)

Unnamed: 0                                                        9002629269
author                     D.L. DavisD.L. DavisD.L. DavisD.L. DavisD.L. D...
statement                  End of eviction moratorium means millions of A...
target                                                                 68911
BinaryNumTarget                                                      68911.0
manual_keywords            Americans, eviction moratoriumAmericans, evict...
tweet                      @POTUS Biden Blunders - 6 Month Update\n\nInfl...
5_label_majority_answer    Mostly AgreeNO MAJORITYAgreeMostly AgreeAgreeA...
3_label_majority_answer    AgreeAgreeAgreeAgreeAgreeAgreeAgreeAgreeAgreeA...
anger                                                                  12384
disgust                                                                16788
fear                                                                    6739
joy                                                                     2825

In [14]:
df1.drop_duplicates(subset=['statement'], inplace=True)


In [15]:
df1['BinaryNumTarget'] = df1['BinaryNumTarget'].astype(int)

In [16]:
emotions_columns = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
df1[emotions_columns] = df1[emotions_columns].astype(float)

In [17]:
df1.dropna(inplace=True)

In [18]:
df1.head(1)

,Unnamed: 0,author,statement,target,BinaryNumTarget,manual_keywords,tweet,5_label_majority_answer,3_label_majority_answer,anger,disgust,fear,joy,neutral,sadness,surprise,textblob,afinn,pattern,vader
0,0,D.L. Davis,End of eviction moratorium means millions of A...,True,1,"Americans, eviction moratorium",@POTUS Biden Blunders - 6 Month Update\n\nInfl...,Mostly Agree,Agree,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,-0.6908


In [19]:
X_text = df1['statement'].values
X_emotions = df1[['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']].values
X_scores = df1[['afinn', 'pattern', 'vader', 'textblob']].values
y = df1['BinaryNumTarget'].values


In [20]:
statements = df1['statement']

max_len_text = max(len(statement.split()) for statement in statements)

print("Maximum length of statements:", max_len_text)

Maximum length of statements: 48


In [21]:
max_len_text = max_len_text  
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_text)
sequences = tokenizer.texts_to_sequences(X_text)
X_text_pad = pad_sequences(sequences, maxlen=max_len_text)

In [22]:
embedding_dim = 100  
vocab_size = len(tokenizer.word_index) + 1


In [23]:
from sklearn.model_selection import StratifiedKFold

In [24]:
def create_model():
    model_text = Sequential()
    model_text.add(Embedding(vocab_size, embedding_dim, input_length=max_len_text))
    model_text.add(Bidirectional(LSTM(128))) 

    model_emotions = Sequential()
    model_emotions.add(Dense(64, input_shape=(7,)))  # 7 features for emotions 
    model_scores = Sequential()
    model_scores.add(Dense(64, input_shape=(4,)))  # 4 features for sentiment scores

    merged = concatenate([model_text.output, model_emotions.output, model_scores.output])
    merged = Dense(64, activation='relu')(merged)
    output = Dense(1, activation='sigmoid')(merged)

    model = tf.keras.models.Model(inputs=[model_text.input, model_emotions.input, model_scores.input], outputs=output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [25]:
n_splits = 5  
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)


In [26]:
for fold, (train_index, test_index) in enumerate(skf.split(X_text_pad, y)):
    print(f"Training Fold {fold+1}...")

    X_train_text, X_test_text = X_text_pad[train_index], X_text_pad[test_index]
    X_train_emotions, X_test_emotions = X_emotions[train_index], X_emotions[test_index]
    X_train_scores, X_test_scores = X_scores[train_index], X_scores[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = create_model()
    model.fit([X_train_text, X_train_emotions, X_train_scores], y_train, epochs=10, batch_size=32, verbose=1)


Training Fold 1...
Epoch 1/10


2023-08-02 02:03:17.528096: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


27/27 [==============================] - 2s 26ms/step - loss: 0.6895 - accuracy: 0.5603
Epoch 2/10
27/27 [==============================] - 1s 32ms/step - loss: 0.6671 - accuracy: 0.6986
Epoch 3/10
27/27 [==============================] - 1s 29ms/step - loss: 0.3771 - accuracy: 0.8522
Epoch 4/10
27/27 [==============================] - 1s 30ms/step - loss: 0.1188 - accuracy: 0.9574
Epoch 5/10
27/27 [==============================] - 1s 27ms/step - loss: 0.0325 - accuracy: 0.9917
Epoch 6/10
27/27 [==============================] - 1s 28ms/step - loss: 0.0091 - accuracy: 1.0000
Epoch 7/10
27/27 [==============================] - 1s 32ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 8/10
27/27 [==============================] - 1s 32ms/step - loss: 7.6014e-04 - accuracy: 1.0000
Epoch 9/10
27/27 [==============================] - 1s 31ms/step - loss: 5.0252e-04 - accuracy: 1.0000
Epoch 10/10
27/27 [==============================] - 1s 29ms/step - loss: 3.7336e-04 - accuracy: 1.0000
Training

In [27]:
    loss, accuracy = model.evaluate([X_test_text, X_test_emotions, X_test_scores], y_test, batch_size=32)
    predictions = model.predict([X_test_text, X_test_emotions, X_test_scores])
    predictions = (predictions > 0.5).astype(int)


7/7 [==============================] - 0s 9ms/step


In [28]:
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

In [29]:
    tn, fp, fn, tp = tf.math.confusion_matrix(y_test, predictions).numpy().ravel()
    accuracy_scores.append((tp + tn) / (tp + tn + fp + fn))
    precision_scores.append(tp / (tp + fp))
    recall_scores.append(tp / (tp + fn))
    f1_scores.append(2 * tp / (2 * tp + fp + fn))

    print(f'Fold {fold+1} - Test Accuracy: {accuracy_scores[-1]:.4f}, Precision: {precision_scores[-1]:.4f}, Recall: {recall_scores[-1]:.4f}, F1-score: {f1_scores[-1]:.4f}')


Fold 5 - Test Accuracy: 0.7393, Precision: 0.7632, Recall: 0.7565, F1-score: 0.7598


In [30]:
print(f'\nAverage Test Accuracy: {np.mean(accuracy_scores):.4f}')
print(f'Average Precision: {np.mean(precision_scores):.4f}')
print(f'Average Recall: {np.mean(recall_scores):.4f}')
print(f'Average F1-score: {np.mean(f1_scores):.4f}')


Average Test Accuracy: 0.7393
Average Precision: 0.7632
Average Recall: 0.7565
Average F1-score: 0.7598


In [31]:
from sklearn.metrics import classification_report

In [32]:
best_fold = np.argmax(accuracy_scores)
X_train_text, X_test_text = X_text_pad[train_index], X_text_pad[test_index]
X_train_emotions, X_test_emotions = X_emotions[train_index], X_emotions[test_index]
X_train_scores, X_test_scores = X_scores[train_index], X_scores[test_index]
y_train, y_test = y[train_index], y[test_index]

best_model = create_model()
best_model.fit([X_train_text, X_train_emotions, X_train_scores], y_train, epochs=10, batch_size=32, verbose=1)


Epoch 1/10
27/27 [==============================] - 2s 27ms/step - loss: 0.6736 - accuracy: 0.5868
Epoch 2/10
27/27 [==============================] - 1s 27ms/step - loss: 0.5557 - accuracy: 0.7096
Epoch 3/10
27/27 [==============================] - 1s 27ms/step - loss: 0.2074 - accuracy: 0.9256
Epoch 4/10
27/27 [==============================] - 1s 28ms/step - loss: 0.0370 - accuracy: 0.9894
Epoch 5/10
27/27 [==============================] - 1s 27ms/step - loss: 0.0081 - accuracy: 0.9988
Epoch 6/10
27/27 [==============================] - 1s 27ms/step - loss: 0.0079 - accuracy: 0.9976
Epoch 7/10
27/27 [==============================] - 1s 30ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 8/10
27/27 [==============================] - 1s 28ms/step - loss: 4.4790e-04 - accuracy: 1.0000
Epoch 9/10
27/27 [==============================] - 1s 31ms/step - loss: 2.9238e-04 - accuracy: 1.0000
Epoch 10/10
27/27 [==============================] - 1s 27ms/step - loss: 2.1739e-04 - accuracy: 1.00

In [33]:
loss, accuracy = best_model.evaluate([X_test_text, X_test_emotions, X_test_scores], y_test, batch_size=32)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

# Make predictions
predictions = best_model.predict([X_test_text, X_test_emotions, X_test_scores])
predictions = (predictions > 0.5).astype(int)

# Print the classification report
print("\nClassification Report:")
print(classification_report(y_test, predictions))

7/7 [==============================] - 0s 9ms/step - loss: 1.5179 - accuracy: 0.7393
Test Loss: 1.5179, Test Accuracy: 0.7393
7/7 [==============================] - 0s 9ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.73      0.72        96
           1       0.77      0.75      0.76       115

    accuracy                           0.74       211
   macro avg       0.74      0.74      0.74       211
weighted avg       0.74      0.74      0.74       211



In [34]:
best_model.save('fake_news_detection_LSTM.h5')

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['statement'], df['BinaryNumTarget'], test_size=0.2, random_state=42)


In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [39]:
# Convert text data into TF-IDF for NB and PAC
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [40]:
# Convert text data into sequences for LSTM
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)
word_index = tokenizer.word_index
X_train_seq = pad_sequences(sequences_train, maxlen=1000)
X_test_seq = pad_sequences(sequences_test, maxlen=1000)

In [42]:
from sklearn.naive_bayes import MultinomialNB


In [43]:
# Train a MultinomialNB classifier
nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)

MultinomialNB()

In [45]:
from sklearn.linear_model import PassiveAggressiveClassifier

In [46]:
# Train a PassiveAggressiveClassifier
pac = PassiveAggressiveClassifier()
pac.fit(X_train_tfidf, y_train)

PassiveAggressiveClassifier()

In [49]:
import numpy as np


In [53]:
# Generate predictions from both models
nb_train_preds = np.expand_dims(nb.predict(X_train_tfidf), axis=-1)
nb_test_preds = np.expand_dims(nb.predict(X_test_tfidf), axis=-1)
pac_train_preds = np.expand_dims(pac.predict(X_train_tfidf), axis=-1)
pac_test_preds = np.expand_dims(pac.predict(X_test_tfidf), axis=-1)

In [54]:
# Create LSTM model
embedding_dim = 100
sequence_input = Input(shape=(1000,), dtype='int32')
nb_input = Input(shape=(1,))
pac_input = Input(shape=(1,))

embedded_sequences = Embedding(len(word_index) + 1, embedding_dim, input_length=1000)(sequence_input)
x = LSTM(128)(embedded_sequences)
x = concatenate([x, Dense(1)(nb_input), Dense(1)(pac_input)])  # Add predictions as features
x = Dense(64, activation='relu')(x)
preds = Dense(1, activation='sigmoid')(x)

model = Model([sequence_input, nb_input, pac_input], preds)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [55]:
# Train the model
model.fit([X_train_seq, nb_train_preds, pac_train_preds], y_train, epochs=10, batch_size=32, validation_data=([X_test_seq, nb_test_preds, pac_test_preds], y_test))

Epoch 1/10
27/27 [==============================] - 14s 489ms/step - loss: 0.5702 - accuracy: 0.8983 - val_loss: 0.5810 - val_accuracy: 0.7547
Epoch 2/10
27/27 [==============================] - 13s 490ms/step - loss: 0.3878 - accuracy: 0.9846 - val_loss: 0.5148 - val_accuracy: 0.7783
Epoch 3/10
27/27 [==============================] - 13s 491ms/step - loss: 0.1475 - accuracy: 0.9586 - val_loss: 0.5504 - val_accuracy: 0.8019
Epoch 4/10
27/27 [==============================] - 13s 499ms/step - loss: 0.0334 - accuracy: 0.9917 - val_loss: 0.7448 - val_accuracy: 0.7689
Epoch 5/10
27/27 [==============================] - 13s 494ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.8319 - val_accuracy: 0.7736
Epoch 6/10
27/27 [==============================] - 13s 499ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 1.0405 - val_accuracy: 0.7689
Epoch 7/10
27/27 [==============================] - 13s 495ms/step - loss: 0.0158 - accuracy: 0.9953 - val_loss: 4.8269 - val_accuracy: 0.4387

In [56]:
from sklearn.metrics import classification_report

In [57]:
# After training the model
y_test_preds = model.predict([X_test_seq, nb_test_preds, pac_test_preds])
y_test_preds = np.round(y_test_preds)  # round the predictions to 0 or 1

7/7 [==============================] - 1s 137ms/step


In [58]:
report = classification_report(y_test, y_test_preds, digits=2)
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.64      0.70        87
           1       0.78      0.86      0.81       125

    accuracy                           0.77       212
   macro avg       0.77      0.75      0.75       212
weighted avg       0.77      0.77      0.77       212



In [59]:
model.save('ensemble_model_LSTM.h5')

In [60]:
import joblib

In [61]:
joblib.dump(nb, 'nb_classifier.joblib')
joblib.dump(pac, 'pac_classifier.joblib')
joblib.dump(vectorizer, 'tfidf_vectorizer.joblib')

['tfidf_vectorizer.joblib']